In [22]:
# Colab setup ------------------
import os, sys, subprocess
# ------------------------------

import numpy as np
import scipy.integrate

import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting

# Set to True to have fully interactive plots with Python;
# Set to False to use pre-built JavaScript-based plots
interactive_python_plots = False
notebook_url = "localhost:8888"

bokeh.io.output_notebook()

def callback(attr, old, new):
    for i in range(len(gata6)):
        ss = scipy.integrate.odeint(rhs, [foxa2_initial, foxf1_initial], t, args=(gata6[i],
                                                                            foxa2_vmax_slider.value,
                                                                            foxf1_vmax_slider.value,
                                                                            gata6_a2_km,
                                                                            gata6_a2_n,
                                                                            foxf1_a2_km_slider.value,
                                                                            foxf1_a2_n_slider.value,
                                                                            foxa2_a2_km_slider.value,
                                                                            foxa2_a2_n_slider.value,
                                                                            gata6_a2_km,
                                                                            gata6_a2_n,
                                                                            foxa2_f1_km_slider.value,
                                                                            foxa2_f1_n_slider.value,
                                                                            foxf1_f1_km_slider.value,
                                                                            foxf1_f1_n_slider.value,))
        foxa2s[i] = ss[-1,0]
        foxf1s[i] = ss[-1,1]
    cds.data["foxa2"] = foxa2s
    cds.data["foxf1"] = foxf1s
    
def foxa2_hill(GATA6, FOXA2, FOXF1, foxa2_max, GATA6_Km, GATA6_n, FOXF1_Km, FOXF1_n, FOXA2_Km, FOXA2_n):
    return foxa2_max * (((GATA6/GATA6_Km) ** GATA6_n) * ((FOXA2/FOXA2_Km) ** FOXA2_n)) / ((1 + (FOXF1/FOXF1_Km) ** FOXF1_n) * (1 + (GATA6/GATA6_Km) ** GATA6_n) * (1 + (FOXA2/FOXA2_Km) ** FOXA2_n)) + 0.1 - 0.2 * FOXA2

def foxf1_hill(GATA6, FOXF1, FOXA2, foxf1_max, GATA6_Km, GATA6_n, FOXA2_Km, FOXA2_n, FOXF1_Km, FOXF1_n):
    return foxf1_max * (((GATA6/GATA6_Km) ** GATA6_n) * ((FOXF1/FOXF1_Km) ** FOXF1_n)) / ((1 + (FOXA2/FOXA2_Km) ** FOXA2_n) * (1 + (GATA6/GATA6_Km) ** GATA6_n) * (1 + (FOXF1/FOXF1_Km) ** FOXF1_n)) + 0.1 - 0.2 * FOXF1

def rhs(X, t, GATA6, foxa2_max, foxf1_max, gata6_a2_km, gata6_a2_n, foxf1_a2_km, foxf1_a2_n, foxa2_a2_km, foxa2_a2_n, 
        gata6_f1_km, gata6_f1_n, foxa2_f1_km, foxa2_f1_n, foxf1_f1_km, fox_f1_f1_n):
    FOXA2, FOXF1 = X
    foxa2 = foxa2_hill(GATA6, FOXA2, FOXF1, foxa2_max, gata6_a2_km, gata6_a2_n, foxf1_a2_km, foxf1_a2_n, foxa2_a2_km, foxa2_a2_n)
    foxf1 = foxf1_hill(GATA6, FOXF1, FOXA2, foxf1_max, gata6_f1_km, gata6_f1_n, foxa2_f1_km, foxa2_f1_n, foxf1_f1_km, fox_f1_f1_n)
    return [foxa2, foxf1]

foxa2_vmax_slider = bokeh.models.Slider(
    title="foxa2 vmax", start=0, end=50, step=0.01, value=12.0, width=150
)
foxf1_vmax_slider = bokeh.models.Slider(
    title="foxf1 vmax", start=0, end=50, step=0.01, value=12.3, width=150
)
foxf1_a2_km_slider = bokeh.models.Slider(
    title="foxf1->a2 Km", start=0.01, end=50, step=0.01, value=0.46, width=150
)
foxf1_a2_n_slider = bokeh.models.Slider(
    title="foxf1->a2 n", start=0, end=50, step=0.1, value=0.20, width=150
)
foxa2_a2_km_slider = bokeh.models.Slider(
    title="foxa2->a2 Km", start=0.01, end=50, step=0.01, value=4.42, width=150
)
foxa2_a2_n_slider = bokeh.models.Slider(
    title="foxa2->a2 n", start=0, end=50, step=0.1, value=1.2, width=150
)
foxf1_f1_km_slider = bokeh.models.Slider(
    title="foxf1->f1 Km", start=0.01, end=50, step=0.01, value=5.41, width=150
)
foxf1_f1_n_slider = bokeh.models.Slider(
    title="foxf1->f1 n", start=0, end=50, step=0.1, value=1.0, width=150
)
foxa2_f1_km_slider = bokeh.models.Slider(
    title="foxa2->f1 Km", start=0.01, end=50, step=0.01, value=6.67, width=150
)
foxa2_f1_n_slider = bokeh.models.Slider(
    title="foxa2->f1 n", start=0, end=50, step=0.1, value=1.0, width=150
)


# Set up time points
gata6_max = 40
gata6 = np.linspace(0, gata6_max, gata6_max+1)
t = np.linspace(0, 720, 720)

# GATA6 specific parameters
gata6_a2_km = 10
gata6_f1_km = 4.5

gata6_a2_n = 2
gata6_f1_n = 1

foxa2_initial = 0
foxf1_initial = 0

foxa2s = np.zeros(len(gata6))
foxf1s = np.zeros(len(gata6))

# Build s, taking slider values are parameters.

for i in range(len(gata6)):
    ss = scipy.integrate.odeint(rhs, [foxa2_initial, foxf1_initial], t, args=(gata6[i],
                                                                        foxa2_vmax_slider.value,
                                                                        foxf1_vmax_slider.value,
                                                                        gata6_a2_km,
                                                                        gata6_a2_n,
                                                                        foxf1_a2_km_slider.value,
                                                                        foxf1_a2_n_slider.value,
                                                                        foxa2_a2_km_slider.value,
                                                                        foxa2_a2_n_slider.value,
                                                                        gata6_a2_km,
                                                                        gata6_a2_n,
                                                                        foxa2_f1_km_slider.value,
                                                                        foxa2_f1_n_slider.value,
                                                                        foxf1_f1_km_slider.value,
                                                                        foxf1_f1_n_slider.value,))
    foxa2s[i] = ss[-1,0]
    foxf1s[i] = ss[-1,1]

# Place the data in a ColumnDataSource
cds = bokeh.models.ColumnDataSource(dict(gata6=gata6, foxa2=foxa2s, foxf1=foxf1s))

# Build the plot
p = bokeh.plotting.figure(
    frame_height=400,
    frame_width=400,
    x_axis_label="GATA6",
    y_axis_label="GATA6 Response",
    x_range=[0, 30],
    y_range=[-0.02, 20],
)
p.line(source=cds, x="gata6", y="foxa2", line_width=2, color='red', legend_label="FOXA2");
p.line(source=cds, x="gata6", y="foxf1", line_width=2, color='green', legend_label="FOXF1")

foxa2_vmax_slider.on_change("value", callback)
foxf1_vmax_slider.on_change("value", callback)
foxf1_a2_km_slider.on_change("value", callback)
foxf1_a2_n_slider.on_change("value", callback)
foxa2_a2_km_slider.on_change("value", callback)
foxa2_a2_n_slider.on_change("value", callback)
foxf1_f1_km_slider.on_change("value", callback)
foxf1_f1_n_slider.on_change("value", callback)
foxa2_f1_km_slider.on_change("value", callback)
foxa2_f1_n_slider.on_change("value", callback)

layout = bokeh.layouts.row(
    p,
    bokeh.models.Spacer(width=30),
    bokeh.layouts.column(foxa2_vmax_slider, foxf1_a2_km_slider, foxf1_a2_n_slider, foxa2_a2_km_slider, foxa2_a2_n_slider),
    bokeh.models.Spacer(width=30),
    bokeh.layouts.column(foxf1_vmax_slider, foxf1_f1_km_slider, foxf1_f1_n_slider, foxa2_f1_km_slider, foxa2_f1_n_slider)
)

def app(doc):
    doc.add_root(layout)

Loading BokehJS ...

In [23]:
interactive_python_plots = True

if interactive_python_plots:
    bokeh.io.show(app, notebook_url=notebook_url)
else:
    bokeh.io.show(biocircuits.jsplots.autorepressor_response_to_pulse())

ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'ModelChanged', 'model': {'id': 'p3124'}, 'attr': 'value', 'new': 23.6}]} 
 error: TypeError('rhs() takes 17 positional arguments but 22 were given')
Traceback (most recent call last):
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 288, in _handle_patch
    message.apply_to_document(self.document, self)
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc

ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'ModelChanged', 'model': {'id': 'p3124'}, 'attr': 'value', 'new': 21.200000000000003}]} 
 error: TypeError('rhs() takes 17 positional arguments but 22 were given')
Traceback (most recent call last):
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 288, in _handle_patch
    message.apply_to_document(self.document, self)
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\protocol\mess

ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'ModelChanged', 'model': {'id': 'p3124'}, 'attr': 'value', 'new': 19.5}]} 
 error: TypeError('rhs() takes 17 positional arguments but 22 were given')
Traceback (most recent call last):
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 288, in _handle_patch
    message.apply_to_document(self.document, self)
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc

ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'ModelChanged', 'model': {'id': 'p3124'}, 'attr': 'value', 'new': 17.5}]} 
 error: TypeError('rhs() takes 17 positional arguments but 22 were given')
Traceback (most recent call last):
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 288, in _handle_patch
    message.apply_to_document(self.document, self)
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc

ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'ModelChanged', 'model': {'id': 'p3124'}, 'attr': 'value', 'new': 16.5}]} 
 error: TypeError('rhs() takes 17 positional arguments but 22 were given')
Traceback (most recent call last):
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 288, in _handle_patch
    message.apply_to_document(self.document, self)
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc

ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'ModelChanged', 'model': {'id': 'p3124'}, 'attr': 'value', 'new': 15.5}]} 
 error: TypeError('rhs() takes 17 positional arguments but 22 were given')
Traceback (most recent call last):
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 288, in _handle_patch
    message.apply_to_document(self.document, self)
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc

ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'ModelChanged', 'model': {'id': 'p3124'}, 'attr': 'value', 'new': 14.2}]} 
 error: TypeError('rhs() takes 17 positional arguments but 22 were given')
Traceback (most recent call last):
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 288, in _handle_patch
    message.apply_to_document(self.document, self)
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc

ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'ModelChanged', 'model': {'id': 'p3124'}, 'attr': 'value', 'new': 10.8}]} 
 error: TypeError('rhs() takes 17 positional arguments but 22 were given')
Traceback (most recent call last):
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 288, in _handle_patch
    message.apply_to_document(self.document, self)
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc

ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'ModelChanged', 'model': {'id': 'p3124'}, 'attr': 'value', 'new': 9.4}]} 
 error: TypeError('rhs() takes 17 positional arguments but 22 were given')
Traceback (most recent call last):
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\server\session.py", line 288, in _handle_patch
    message.apply_to_document(self.document, self)
  File "C:\Users\ajin40\AppData\Local\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc.

In [32]:
# Colab setup ------------------
import os, sys, subprocess
# ------------------------------

import numpy as np
import scipy.integrate

import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting

# Set to True to have fully interactive plots with Python;
# Set to False to use pre-built JavaScript-based plots
interactive_python_plots = False
notebook_url = "localhost:8888"

bokeh.io.output_notebook()

def callback(attr, old, new):
    for i in range(len(gata6)):
        for j in range(len(nanog)):
            ss = scipy.integrate.odeint(rhs, [foxa2_initial, foxf1_initial], t, args=(gata6[i],
                                                                                      nanog[i],
                                                                                    foxa2_vmax_slider.value,
                                                                                    foxf1_vmax_slider.value,
                                                                                    gata6_a2_km,
                                                                                    gata6_a2_n,
                                                                                    foxf1_a2_km_slider.value,
                                                                                    foxf1_a2_n_slider.value,
                                                                                    foxa2_a2_km_slider.value,
                                                                                    foxa2_a2_n_slider.value,
                                                                                    gata6_a2_km,
                                                                                    gata6_a2_n,
                                                                                    foxa2_f1_km_slider.value,
                                                                                    foxa2_f1_n_slider.value,
                                                                                    foxf1_f1_km_slider.value,
                                                                                    foxf1_f1_n_slider.value,
                                                                                     nanog_a2_km_slider.value, 
                                                                                     nanog_a2_n_slider.value,
                                                                                     nanog_f1_km_slider.value,
                                                                                     nanog_f1_n_slider.value))
            foxa2s[i, j] = ss[-1,0]
            foxf1s[i, j] = ss[-1,1]
    a2s_data.data["image"] = [foxa2s]
    f1s_data.data["image"] = [foxf1s]

    
def foxa2_hill(GATA6, NANOG, FOXA2, FOXF1, foxa2_max, GATA6_Km, GATA6_n, FOXF1_Km, FOXF1_n, FOXA2_Km, FOXA2_n ,NANOG_Km, NANOG_n):
    return foxa2_max * (((GATA6/GATA6_Km) ** GATA6_n) * ((FOXA2/FOXA2_Km) ** FOXA2_n)) / ((1 + (NANOG/NANOG_Km) ** NANOG_n) * (1 + (FOXF1/FOXF1_Km) ** FOXF1_n) * (1 + (GATA6/GATA6_Km) ** GATA6_n) * (1 + (FOXA2/FOXA2_Km) ** FOXA2_n)) + 0.1 - 0.2 * FOXA2

def foxf1_hill(GATA6, NANOG, FOXF1, FOXA2, foxf1_max, GATA6_Km, GATA6_n, FOXA2_Km, FOXA2_n, FOXF1_Km, FOXF1_n, NANOG_Km, NANOG_n):
    return foxf1_max * (((GATA6/GATA6_Km) ** GATA6_n) * ((FOXF1/FOXF1_Km) ** FOXF1_n)) / ((1 + (NANOG/NANOG_Km) ** NANOG_n) * (1 + (FOXA2/FOXA2_Km) ** FOXA2_n) * (1 + (GATA6/GATA6_Km) ** GATA6_n) * (1 + (FOXF1/FOXF1_Km) ** FOXF1_n)) + 0.1 - 0.2 * FOXF1

def rhs(X, t, GATA6, NANOG, foxa2_max, foxf1_max, gata6_a2_km, gata6_a2_n, foxf1_a2_km, foxf1_a2_n, foxa2_a2_km, foxa2_a2_n, 
        gata6_f1_km, gata6_f1_n, foxa2_f1_km, foxa2_f1_n, foxf1_f1_km, fox_f1_f1_n, nanog_a2_km, nanog_a2_n, nanog_f1_km, nanog_f1_n):
    FOXA2, FOXF1 = X
    foxa2 = foxa2_hill(GATA6, NANOG, FOXA2, FOXF1, foxa2_max, gata6_a2_km, gata6_a2_n, foxf1_a2_km, foxf1_a2_n, foxa2_a2_km, foxa2_a2_n, nanog_a2_km, nanog_a2_n)
    foxf1 = foxf1_hill(GATA6, NANOG, FOXF1, FOXA2, foxf1_max, gata6_f1_km, gata6_f1_n, foxa2_f1_km, foxa2_f1_n, foxf1_f1_km, fox_f1_f1_n, nanog_f1_km, nanog_f1_n)
    return [foxa2, foxf1]

foxa2_vmax_slider = bokeh.models.Slider(
    title="foxa2 vmax", start=0, end=50, step=0.01, value=12.0, width=150
)
foxf1_vmax_slider = bokeh.models.Slider(
    title="foxf1 vmax", start=0, end=50, step=0.01, value=12.3, width=150
)
foxf1_a2_km_slider = bokeh.models.Slider(
    title="foxf1->a2 Km", start=0.01, end=50, step=0.01, value=0.46, width=150
)
foxf1_a2_n_slider = bokeh.models.Slider(
    title="foxf1->a2 n", start=0, end=50, step=0.1, value=0.20, width=150
)
foxa2_a2_km_slider = bokeh.models.Slider(
    title="foxa2->a2 Km", start=0.01, end=50, step=0.01, value=4.42, width=150
)
foxa2_a2_n_slider = bokeh.models.Slider(
    title="foxa2->a2 n", start=0, end=50, step=0.1, value=1.2, width=150
)
foxf1_f1_km_slider = bokeh.models.Slider(
    title="foxf1->f1 Km", start=0.01, end=50, step=0.01, value=5.41, width=150
)
foxf1_f1_n_slider = bokeh.models.Slider(
    title="foxf1->f1 n", start=0, end=50, step=0.1, value=1.0, width=150
)
foxa2_f1_km_slider = bokeh.models.Slider(
    title="foxa2->f1 Km", start=0.01, end=50, step=0.01, value=6.67, width=150
)
foxa2_f1_n_slider = bokeh.models.Slider(
    title="foxa2->f1 n", start=0, end=50, step=0.1, value=1.0, width=150
)
nanog_f1_km_slider = bokeh.models.Slider(
    title="nanog->f1 Km", start=0.01, end=50, step=0.01, value=0.5, width=150
)
nanog_f1_n_slider = bokeh.models.Slider(
    title="nanog->f1 n", start=0, end=50, step=0.1, value=1.0, width=150
)
nanog_a2_km_slider = bokeh.models.Slider(
    title="nanog->a2 Km", start=0.01, end=50, step=0.01, value=0.5, width=150
)
nanog_a2_n_slider = bokeh.models.Slider(
    title="nanog->a2 n", start=0, end=50, step=0.1, value=2.0, width=150
)

# Set up time points
gata6_max = 40
gata6 = np.linspace(0, gata6_max, gata6_max+1)
nanog = np.linspace(0, gata6_max, gata6_max+1)
t = np.linspace(0, 720, 720)

# GATA6 specific parameters
gata6_a2_km = 10
gata6_f1_km = 4.5

gata6_a2_n = 2
gata6_f1_n = 1

foxa2_initial = 0
foxf1_initial = 0

foxa2s = np.zeros((len(gata6), len(nanog)))
foxf1s = np.zeros((len(gata6), len(nanog)))

# Build s, taking slider values are parameters.

for i in range(len(gata6)):
    for j in range(len(nanog)):
        ss = scipy.integrate.odeint(rhs, [foxa2_initial, foxf1_initial], t, args=(gata6[i],
                                                                                  nanog[i],
                                                                                foxa2_vmax_slider.value,
                                                                                foxf1_vmax_slider.value,
                                                                                gata6_a2_km,
                                                                                gata6_a2_n,
                                                                                foxf1_a2_km_slider.value,
                                                                                foxf1_a2_n_slider.value,
                                                                                foxa2_a2_km_slider.value,
                                                                                foxa2_a2_n_slider.value,
                                                                                gata6_a2_km,
                                                                                gata6_a2_n,
                                                                                foxa2_f1_km_slider.value,
                                                                                foxa2_f1_n_slider.value,
                                                                                foxf1_f1_km_slider.value,
                                                                                foxf1_f1_n_slider.value,
                                                                                 nanog_a2_km_slider.value, 
                                                                                 nanog_a2_n_slider.value,
                                                                                 nanog_f1_km_slider.value,
                                                                                 nanog_f1_n_slider.value))
        foxa2s[i, j] = ss[-1,0]
        foxf1s[i, j] = ss[-1,1]

# # Place the data in a ColumnDataSource
a2s_data = bokeh.models.ColumnDataSource(dict(image=[foxa2s]))
f1s_data = bokeh.models.ColumnDataSource(dict(image=[foxf1s]))

# Build the plot
a2 = bokeh.plotting.figure(
    frame_height=300,
    frame_width=300,
    x_axis_label="GATA6",
    y_axis_label="NANOG",
    x_range=[-0.02, 40],
    y_range=[-0.02, 40],
)
# Build the plot
f1 = bokeh.plotting.figure(
    frame_height=300,
    frame_width=300,
    x_axis_label="GATA6",
    y_axis_label="NANOG",
    x_range=[-0.02, 40],
    y_range=[-0.02, 40],
)
a2_image = a2.image(source=a2s_data, image='image', x=0, y=0, dw=40, dh=40, palette='Reds9');
f1_image = f1.image(source=f1s_data, image='image', x=0, y=0, dw=40, dh=40, palette='Greens9');
a2_color_bar = a2_image.construct_color_bar(width=10)
f1_color_bar = f1_image.construct_color_bar(width=10)
a2.add_layout(a2_color_bar, 'left')
f1.add_layout(f1_color_bar, 'right')

foxa2_vmax_slider.on_change("value", callback)
foxf1_vmax_slider.on_change("value", callback)
foxf1_a2_km_slider.on_change("value", callback)
foxf1_a2_n_slider.on_change("value", callback)
foxa2_a2_km_slider.on_change("value", callback)
foxa2_a2_n_slider.on_change("value", callback)
foxf1_f1_km_slider.on_change("value", callback)
foxf1_f1_n_slider.on_change("value", callback)
foxa2_f1_km_slider.on_change("value", callback)
foxa2_f1_n_slider.on_change("value", callback)
nanog_a2_km_slider.on_change("value", callback)
nanog_a2_n_slider.on_change("value", callback)
nanog_f1_km_slider.on_change("value", callback)
nanog_f1_n_slider.on_change("value", callback)


layout = bokeh.layouts.column(bokeh.layouts.row(
    a2,
    f1),
    bokeh.layouts.row(bokeh.models.Spacer(width=30),
    bokeh.layouts.column(foxa2_vmax_slider, foxf1_a2_km_slider, foxf1_a2_n_slider, foxa2_a2_km_slider, foxa2_a2_n_slider),
    bokeh.models.Spacer(width=30),
    bokeh.layouts.column(foxf1_vmax_slider, foxf1_f1_km_slider, foxf1_f1_n_slider, foxa2_f1_km_slider, foxa2_f1_n_slider),
    bokeh.models.Spacer(width=30),
    bokeh.layouts.column(nanog_a2_km_slider, nanog_a2_n_slider, nanog_f1_km_slider, nanog_f1_n_slider))
)

def new_app(doc):
    doc.add_root(layout)

Loading BokehJS ...

In [33]:
interactive_python_plots = True

if interactive_python_plots:
    bokeh.io.show(new_app, notebook_url=notebook_url)
else:
    bokeh.io.show(biocircuits.jsplots.autorepressor_response_to_pulse())

In [ ]:
# Colab setup ------------------
import os, sys, subprocess
# ------------------------------

import numpy as np
import scipy.integrate

import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting

# Set to True to have fully interactive plots with Python;
# Set to False to use pre-built JavaScript-based plots
interactive_python_plots = False
notebook_url = "localhost:8888"

bokeh.io.output_notebook()

def callback(attr, old, new):
    for i in range(len(gata6)):
        for j in range(len(nanog)):
            ss = scipy.integrate.odeint(rhs, [foxa2_initial, foxf1_initial], t, args=(gata6[i],
                                                                                      nanog[i],
                                                                                    foxa2_vmax_slider.value,
                                                                                    foxf1_vmax_slider.value,
                                                                                    gata6_a2_km,
                                                                                    gata6_a2_n,
                                                                                    foxf1_a2_km_slider.value,
                                                                                    foxf1_a2_n_slider.value,
                                                                                    foxa2_a2_km_slider.value,
                                                                                    foxa2_a2_n_slider.value,
                                                                                    gata6_a2_km,
                                                                                    gata6_a2_n,
                                                                                    foxa2_f1_km_slider.value,
                                                                                    foxa2_f1_n_slider.value,
                                                                                    foxf1_f1_km_slider.value,
                                                                                    foxf1_f1_n_slider.value,
                                                                                     nanog_a2_km_slider.value, 
                                                                                     nanog_a2_n_slider.value,
                                                                                     nanog_f1_km_slider.value,
                                                                                     nanog_f1_n_slider.value))
            foxa2s[i, j] = ss[-1,0]
            foxf1s[i, j] = ss[-1,1]
    a2s_data.data["image"] = [foxa2s]
    f1s_data.data["image"] = [foxf1s]

    
def foxa2_hill(GATA6, NANOG, FOXA2, FOXF1, foxa2_max, GATA6_Km, GATA6_n, FOXF1_Km, FOXF1_n, FOXA2_Km, FOXA2_n ,NANOG_Km, NANOG_n):
    return foxa2_max * (((GATA6/GATA6_Km) ** GATA6_n) * ((FOXA2/FOXA2_Km) ** FOXA2_n)) / ((1 + (NANOG/NANOG_Km) ** NANOG_n) * (1 + (FOXF1/FOXF1_Km) ** FOXF1_n) * (1 + (GATA6/GATA6_Km) ** GATA6_n) * (1 + (FOXA2/FOXA2_Km) ** FOXA2_n)) + 0.1 - 0.2 * FOXA2

def foxf1_hill(GATA6, NANOG, FOXF1, FOXA2, foxf1_max, GATA6_Km, GATA6_n, FOXA2_Km, FOXA2_n, FOXF1_Km, FOXF1_n, NANOG_Km, NANOG_n):
    return foxf1_max * (((GATA6/GATA6_Km) ** GATA6_n) * ((FOXF1/FOXF1_Km) ** FOXF1_n)) / ((1 + (NANOG/NANOG_Km) ** NANOG_n) * (1 + (FOXA2/FOXA2_Km) ** FOXA2_n) * (1 + (GATA6/GATA6_Km) ** GATA6_n) * (1 + (FOXF1/FOXF1_Km) ** FOXF1_n)) + 0.1 - 0.2 * FOXF1

def rhs(X, t, GATA6, NANOG, foxa2_max, foxf1_max, gata6_a2_km, gata6_a2_n, foxf1_a2_km, foxf1_a2_n, foxa2_a2_km, foxa2_a2_n, 
        gata6_f1_km, gata6_f1_n, foxa2_f1_km, foxa2_f1_n, foxf1_f1_km, fox_f1_f1_n, nanog_a2_km, nanog_a2_n, nanog_f1_km, nanog_f1_n):
    FOXA2, FOXF1 = X
    foxa2 = foxa2_hill(GATA6, NANOG, FOXA2, FOXF1, foxa2_max, gata6_a2_km, gata6_a2_n, foxf1_a2_km, foxf1_a2_n, foxa2_a2_km, foxa2_a2_n, nanog_a2_km, nanog_a2_n)
    foxf1 = foxf1_hill(GATA6, NANOG, FOXF1, FOXA2, foxf1_max, gata6_f1_km, gata6_f1_n, foxa2_f1_km, foxa2_f1_n, foxf1_f1_km, fox_f1_f1_n, nanog_f1_km, nanog_f1_n)
    return [foxa2, foxf1]

foxa2_vmax_slider = bokeh.models.Slider(
    title="foxa2 vmax", start=0, end=50, step=0.01, value=12.0, width=150
)
foxf1_vmax_slider = bokeh.models.Slider(
    title="foxf1 vmax", start=0, end=50, step=0.01, value=12.3, width=150
)
foxf1_a2_km_slider = bokeh.models.Slider(
    title="foxf1->a2 Km", start=0.01, end=50, step=0.01, value=0.46, width=150
)
foxf1_a2_n_slider = bokeh.models.Slider(
    title="foxf1->a2 n", start=0, end=50, step=0.1, value=0.20, width=150
)
foxa2_a2_km_slider = bokeh.models.Slider(
    title="foxa2->a2 Km", start=0.01, end=50, step=0.01, value=4.42, width=150
)
foxa2_a2_n_slider = bokeh.models.Slider(
    title="foxa2->a2 n", start=0, end=50, step=0.1, value=1.2, width=150
)
foxf1_f1_km_slider = bokeh.models.Slider(
    title="foxf1->f1 Km", start=0.01, end=50, step=0.01, value=5.41, width=150
)
foxf1_f1_n_slider = bokeh.models.Slider(
    title="foxf1->f1 n", start=0, end=50, step=0.1, value=1.0, width=150
)
foxa2_f1_km_slider = bokeh.models.Slider(
    title="foxa2->f1 Km", start=0.01, end=50, step=0.01, value=6.67, width=150
)
foxa2_f1_n_slider = bokeh.models.Slider(
    title="foxa2->f1 n", start=0, end=50, step=0.1, value=1.0, width=150
)
nanog_f1_km_slider = bokeh.models.Slider(
    title="nanog->f1 Km", start=0.01, end=50, step=0.01, value=0.5, width=150
)
nanog_f1_n_slider = bokeh.models.Slider(
    title="nanog->f1 n", start=0, end=50, step=0.1, value=1.0, width=150
)
nanog_a2_km_slider = bokeh.models.Slider(
    title="nanog->a2 Km", start=0.01, end=50, step=0.01, value=0.5, width=150
)
nanog_a2_n_slider = bokeh.models.Slider(
    title="nanog->a2 n", start=0, end=50, step=0.1, value=2.0, width=150
)

# GATA6 specific parameters
gata6_a2_km = bokeh.models.Slider(
    title="gata6->a2 Km", start=0.01, end=50, step=0.01, value=10, width=150
)
gata6_f1_km = bokeh.models.Slider(
    title="gata6->a2 n", start=0.01, end=50, step=0.1, value=4.5, width=150
)
gata6_a2_n = bokeh.models.Slider(
    title="gata6->f1 Km", start=0.01, end=50, step=0.01, value=2, width=150
)
gata6_f1_n = bokeh.models.Slider(
    title="gata6->f1 n", start=0.01, end=50, step=0.1, value=1, width=150
)
# Set up time points
gata6_max = 40
gata6 = np.linspace(0, gata6_max, gata6_max+1)
nanog = np.linspace(0, gata6_max, gata6_max+1)
t = np.linspace(0, 720, 720)


foxa2_initial = 0
foxf1_initial = 0

foxa2s = np.zeros((len(gata6), len(nanog)))
foxf1s = np.zeros((len(gata6), len(nanog)))

# Build s, taking slider values are parameters.

for i in range(len(gata6)):
    for j in range(len(nanog)):
        ss = scipy.integrate.odeint(rhs, [foxa2_initial, foxf1_initial], t, args=(gata6[i],
                                                                                  nanog[i],
                                                                                foxa2_vmax_slider.value,
                                                                                foxf1_vmax_slider.value,
                                                                                gata6_a2_km.value,
                                                                                gata6_a2_n.value,
                                                                                foxf1_a2_km_slider.value,
                                                                                foxf1_a2_n_slider.value,
                                                                                foxa2_a2_km_slider.value,
                                                                                foxa2_a2_n_slider.value,
                                                                                gata6_a2_km.value,
                                                                                gata6_a2_n.value,
                                                                                foxa2_f1_km_slider.value,
                                                                                foxa2_f1_n_slider.value,
                                                                                foxf1_f1_km_slider.value,
                                                                                foxf1_f1_n_slider.value,
                                                                                 nanog_a2_km_slider.value, 
                                                                                 nanog_a2_n_slider.value,
                                                                                 nanog_f1_km_slider.value,
                                                                                 nanog_f1_n_slider.value))
        foxa2s[i, j] = ss[-1,0]
        foxf1s[i, j] = ss[-1,1]

# # Place the data in a ColumnDataSource
a2s_data = bokeh.models.ColumnDataSource(dict(image=[foxa2s]))
f1s_data = bokeh.models.ColumnDataSource(dict(image=[foxf1s]))

# Build the plot
a2 = bokeh.plotting.figure(
    frame_height=300,
    frame_width=300,
    x_axis_label="GATA6",
    y_axis_label="NANOG",
    x_range=[-0.02, 40],
    y_range=[-0.02, 40],
)
# Build the plot
f1 = bokeh.plotting.figure(
    frame_height=300,
    frame_width=300,
    x_axis_label="GATA6",
    y_axis_label="NANOG",
    x_range=[-0.02, 40],
    y_range=[-0.02, 40],
)
a2_image = a2.image(source=a2s_data, image='image', x=0, y=0, dw=40, dh=40, palette='Reds9');
f1_image = f1.image(source=f1s_data, image='image', x=0, y=0, dw=40, dh=40, palette='Greens9');
a2_color_bar = a2_image.construct_color_bar(width=10)
f1_color_bar = f1_image.construct_color_bar(width=10)
a2.add_layout(a2_color_bar, 'left')
f1.add_layout(f1_color_bar, 'right')

foxa2_vmax_slider.on_change("value", callback)
foxf1_vmax_slider.on_change("value", callback)
foxf1_a2_km_slider.on_change("value", callback)
foxf1_a2_n_slider.on_change("value", callback)
foxa2_a2_km_slider.on_change("value", callback)
foxa2_a2_n_slider.on_change("value", callback)
foxf1_f1_km_slider.on_change("value", callback)
foxf1_f1_n_slider.on_change("value", callback)
foxa2_f1_km_slider.on_change("value", callback)
foxa2_f1_n_slider.on_change("value", callback)
nanog_a2_km_slider.on_change("value", callback)
nanog_a2_n_slider.on_change("value", callback)
nanog_f1_km_slider.on_change("value", callback)
nanog_f1_n_slider.on_change("value", callback)
gata6_a2_km_slider.on_change("value", callback)
gata6_a2_n_slider.on_change("value", callback)
gata6_f1_km_slider.on_change("value", callback)
gata6_f1_n_slider.on_change("value", callback)


layout = bokeh.layouts.column(bokeh.layouts.row(
    a2,
    f1),
    bokeh.layouts.row(bokeh.models.Spacer(width=30),
    bokeh.layouts.column(foxa2_vmax_slider, foxf1_a2_km_slider, foxf1_a2_n_slider, foxa2_a2_km_slider, foxa2_a2_n_slider),
    bokeh.models.Spacer(width=30),
    bokeh.layouts.column(foxf1_vmax_slider, foxf1_f1_km_slider, foxf1_f1_n_slider, foxa2_f1_km_slider, foxa2_f1_n_slider),
    bokeh.models.Spacer(width=30),
    bokeh.layouts.column(nanog_a2_km_slider, nanog_a2_n_slider, nanog_f1_km_slider, nanog_f1_n_slider))
)

def new_app(doc):
    doc.add_root(layout)